In [8]:
import sys
import os
import logging
sys.path.append(r"..")
from utils import *
from model import *
import numpy as np
import pandas as pd
import deepctr
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
import lightgbm as lgb
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score,roc_auc_score,f1_score,recall_score
from gensim import models
from scipy import sparse
from scipy.sparse import csr_matrix, coo_matrix

from collections import defaultdict
from collections import Counter

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.python.client import device_lib

print(tf.__version__)
print(tf.test.is_built_with_gpu_support)
print(tf.test.is_gpu_available())
print(device_lib.list_local_devices())
os.environ["CUDA_VISIBLE_DEVICES"] = '0' #use GPU with ID=0
#os.environ["CUDA_VISIBLE_DEVICES"]= '-1'
gpus = tf.config.experimental.list_physical_devices('GPU')
#tf.config.experimental.set_memory_growth(gpus[0], True)
#对需要进行限制的GPU进行设置
tf.config.experimental.set_virtual_device_configuration(gpus[0],
                                                      [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=4096)])
gpus

2.1.1
<function is_built_with_gpu_support at 0x0000018C2256FAF8>
Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
True
[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 6520182559309426620
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 4930941747
locality {
  bus_id: 1
  links {
  }
}
incarnation: 9053664380288170245
physical_device_desc: "device: 0, name: GeForce GTX 1060 6GB, pci bus id: 0000:01:00.0, compute capability: 6.1"
]


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [9]:
user_ids = pd.read_pickle(f"{pickle_path}/user_ids_relencode.pickle")
user = pd.read_csv(train_preliminary_p + "user.csv", encoding='utf-8')
user_ids = user_ids.merge(user, how='left', on='user_id')
del user
print(user_ids.shape)
user_ids.head(2)

(1900000, 17)


,user_id,creative_id,ad_id,product_id,product_category,advertiser_id,industry,click_times,time,creative_id_len,ad_id_len,product_id_len,product_category_len,advertiser_len,industry_len,age,gender
0,1,"[642275, 170988, 685687, 1307085, 100774, 5953...","[581171, 156554, 620126, 1166887, 92441, 55685...","[1, 125, 1, 1, 1132, 1, 1, 1, 1, 1244, 55, 55,...","[5, 2, 5, 5, 2, 18, 5, 5, 18, 2, 2, 2, 2]","[6411, 8522, 26781, 12996, 10113, 13007, 15341...","[323, 7, 104, 323, 1, 323, 73, 214, 64, 235, 2...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2]","[20, 20, 20, 39, 40, 43, 46, 52, 60, 64, 64, 7...",12,12,6,3,12,9,4.0,1.0
1,2,"[52634, 128035, 32734, 478744, 10372, 349730, ...","[49388, 117016, 31581, 435462, 11432, 318236, ...","[76, 69, 118, 118, 1193, 76, 1, 1064, 1, 1, 10...","[2, 2, 2, 2, 2, 2, 18, 2, 18, 18, 2, 18, 2, 18...","[20681, 9438, 16619, 23545, 705, 20681, 31504,...","[315, 235, 7, 7, 314, 315, 47, 7, 47, 47, 239,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[10, 11, 14, 17, 28, 28, 28, 38, 38, 39, 41, 4...",42,42,20,3,36,15,10.0,1.0


In [10]:
creative_id_v = models.KeyedVectors.load_word2vec_format(f"{path_save}/creative_id_w2v_128.bin", binary=True)
ad_id_v = models.KeyedVectors.load_word2vec_format(f"{path_save}/ad_id_w2v_128.bin", binary=True)
advertiser_id_v = models.KeyedVectors.load_word2vec_format(f"{path_save}/advertiser_id_w2v_64.bin", binary=True)
product_id_v = models.KeyedVectors.load_word2vec_format(f"{path_save}/product_id_w2v_64.bin", binary=True)

industry_v = models.KeyedVectors.load_word2vec_format(f"{path_save}/industry_w2v_32.bin", binary=True)
product_category_v = models.KeyedVectors.load_word2vec_format(f"{path_save}/product_category_w2v_16.bin", binary=True)
click_times_v = models.KeyedVectors.load_word2vec_format(f"{path_save}/click_times_w2v_8.bin", binary=True)
time_v = models.KeyedVectors.load_word2vec_format(f"{path_save}/time_w2v_16.bin", binary=True)


In [11]:
creative_id_vs = len(creative_id_v.vocab) + 1
ad_id_vs = len(ad_id_v.vocab) + 1
advertiser_id_vs = len(advertiser_id_v.vocab) + 1
product_id_vs = len(product_id_v.vocab) + 1

industry_vs = len(industry_v.vocab) + 1
product_category_vs = len(product_category_v.vocab) + 1
click_times_vs = len(click_times_v.vocab) + 1
time_vs = len(time_v.vocab) + 1

In [12]:
creative_id_em = np.zeros((creative_id_vs, creative_id_v.vector_size))
ad_id_em = np.zeros((ad_id_vs, ad_id_v.vector_size))
advertiser_id_em = np.zeros((advertiser_id_vs, advertiser_id_v.vector_size))
product_id_em = np.zeros((product_id_vs, product_id_v.vector_size))

industry_em = np.zeros((industry_vs, industry_v.vector_size))
product_category_em = np.zeros((product_category_vs, product_category_v.vector_size))
click_times_em = np.zeros((click_times_vs, click_times_v.vector_size))
time_em = np.zeros((time_vs, time_v.vector_size))


for w in creative_id_v.vocab:
    creative_id_em[int(w)] = creative_id_v[w]
for w in ad_id_v.vocab:
    ad_id_em[int(w)] = ad_id_v[w]    
for w in advertiser_id_v.vocab:
    advertiser_id_em[int(w)] = advertiser_id_v[w]
for w in product_id_v.vocab:
    product_id_em[int(w)] = product_id_v[w]
    
for w in industry_v.vocab:
    industry_em[int(w)] = industry_v[w]
for w in product_category_v.vocab:
    product_category_em[int(w)] = product_category_v[w]    
for w in click_times_v.vocab:
    click_times_em[int(w)] = click_times_v[w]
for w in time_v.vocab:
    time_em[int(w)] = time_v[w]
    
    
creative_id_em.shape,ad_id_em.shape,advertiser_id_em.shape,product_id_em.shape

((3412768, 128), (3027359, 128), (57871, 64), (39058, 64))

In [13]:
industry_em.shape,product_category_em.shape,click_times_em.shape,time_em.shape

((333, 32), (19, 16), (9, 8), (92, 16))

In [14]:
# 超参数
max_length = 200
embedding_dim = 128
units = 128
num_classes = 2
batch_size = 256
epochs = 2

In [15]:
creative_id_train_seq = keras.preprocessing.sequence.pad_sequences(user_ids['creative_id'][:810000],value = 0,padding = 'post',maxlen = max_length )
creative_id_val_seq = keras.preprocessing.sequence.pad_sequences(user_ids['creative_id'][810000:900000],value = 0,padding = 'post',maxlen = max_length )
creative_id_test_seq = keras.preprocessing.sequence.pad_sequences(user_ids['creative_id'][900000:],value = 0,padding = 'post',maxlen = max_length )

ad_id_train_seq = keras.preprocessing.sequence.pad_sequences(user_ids['ad_id'][:810000],value = 0,padding = 'post',maxlen = max_length )
ad_id_val_seq = keras.preprocessing.sequence.pad_sequences(user_ids['ad_id'][810000:900000],value = 0,padding = 'post',maxlen = max_length )
ad_id_test_seq = keras.preprocessing.sequence.pad_sequences(user_ids['ad_id'][900000:],value = 0,padding = 'post',maxlen = max_length )

advertiser_id_train_seq = keras.preprocessing.sequence.pad_sequences(user_ids['advertiser_id'][:810000],value = 0,padding = 'post',maxlen = max_length )
advertiser_id_val_seq = keras.preprocessing.sequence.pad_sequences(user_ids['advertiser_id'][810000:900000],value = 0,padding = 'post',maxlen = max_length )
advertiser_id_test_seq = keras.preprocessing.sequence.pad_sequences(user_ids['advertiser_id'][900000:],value = 0,padding = 'post',maxlen = max_length )

product_id_train_seq = keras.preprocessing.sequence.pad_sequences(user_ids['product_id'][:810000],value = 0,padding = 'post',maxlen = max_length )
product_id_val_seq = keras.preprocessing.sequence.pad_sequences(user_ids['product_id'][810000:900000],value = 0,padding = 'post',maxlen = max_length )
product_id_test_seq = keras.preprocessing.sequence.pad_sequences(user_ids['product_id'][900000:],value = 0,padding = 'post',maxlen = max_length )


industry_train_seq = keras.preprocessing.sequence.pad_sequences(user_ids['industry'][:810000],value = 0,padding = 'post',maxlen = max_length )
industry_val_seq = keras.preprocessing.sequence.pad_sequences(user_ids['industry'][810000:900000],value = 0,padding = 'post',maxlen = max_length )
industry_test_seq = keras.preprocessing.sequence.pad_sequences(user_ids['industry'][900000:],value = 0,padding = 'post',maxlen = max_length )


product_category_train_seq = keras.preprocessing.sequence.pad_sequences(user_ids['product_category'][:810000],value = 0,padding = 'post',maxlen = max_length )
product_category_val_seq = keras.preprocessing.sequence.pad_sequences(user_ids['product_category'][810000:900000],value = 0,padding = 'post',maxlen = max_length )
product_category_test_seq = keras.preprocessing.sequence.pad_sequences(user_ids['product_category'][900000:],value = 0,padding = 'post',maxlen = max_length )


click_times_train_seq = keras.preprocessing.sequence.pad_sequences(user_ids['click_times'][:810000],value = 0,padding = 'post',maxlen = max_length )
click_times_val_seq = keras.preprocessing.sequence.pad_sequences(user_ids['click_times'][810000:900000],value = 0,padding = 'post',maxlen = max_length )
click_times_test_seq = keras.preprocessing.sequence.pad_sequences(user_ids['click_times'][900000:],value = 0,padding = 'post',maxlen = max_length )


time_train_seq = keras.preprocessing.sequence.pad_sequences(user_ids['time'][:810000],value = 0,padding = 'post',maxlen = max_length )
time_val_seq = keras.preprocessing.sequence.pad_sequences(user_ids['time'][810000:900000],value = 0,padding = 'post',maxlen = max_length )
time_test_seq = keras.preprocessing.sequence.pad_sequences(user_ids['time'][900000:],value = 0,padding = 'post',maxlen = max_length )


In [7]:
# # 放入cpu中
# with tf.device("/CPU:0"):
#     creative_id_em = tf.constant(creative_id_em)
#     ad_id_em = tf.constant(ad_id_em)

In [16]:
gender_train_label = np.array(user_ids['gender'][:810000])
gender_val_label = np.array(user_ids['gender'][810000:900000])

In [17]:
age_train_label = np.array(user_ids['age'][:810000])
age_val_label = np.array(user_ids['age'][810000:900000])

In [10]:
# def lstm_model(max_len,em_dim, units, num_class):
#     inputs = keras.Input(shape=(max_len,em_dim), name='input')
#     lstm_output = keras.layers.Bidirectional(keras.layers.LSTM(units))(inputs)
#     fc = keras.layers.Dense(units, activation='relu')(lstm_output)
#     outputs = keras.layers.Dense(num_class, activation='softmax')(fc)
#     model = keras.Model(inputs=inputs, outputs=outputs)
#     return model

In [11]:
# def input_fn(feature_dict, label=None, epochs=5, shuffle=True, batch_size=64, fit_key='train'):
#     if fit_key == 'train':
#         dataset = tf.data.Dataset.from_tensor_slices((feature_dict, label))
#         dataset = dataset.map(lambda X,y: (tf.concat([tf.nn.embedding_lookup(creative_id_em, X['creative_id']),
#                                                        tf.nn.embedding_lookup(creative_id_em, X['ad_id']),
#                                                       ], axis=-1),y),
#                               num_parallel_calls=tf.data.experimental.AUTOTUNE)
#     else:
#         dataset = tf.data.Dataset.from_tensor_slices((feature_dict))
#         dataset = dataset.map(lambda X,y: (tf.concat([tf.nn.embedding_lookup(creative_id_em, X['creative_id']),
#                                                        tf.nn.embedding_lookup(creative_id_em, X['ad_id']),
#                                                       ], axis=-1)),
#                               num_parallel_calls=tf.data.experimental.AUTOTUNE)
        
    
#     if shuffle:
#         dataset = dataset.shuffle(100*batch_size)
#     dataset = dataset.repeat(epochs).batch(batch_size)
#     return dataset

In [12]:
# model = lstm_model(max_len=200,
#                    em_dim=256,
#                    units=128,
#                    num_class=2)
# model.compile(optimizer = keras.optimizers.Adam(0.001),
#               loss = keras.losses.sparse_categorical_crossentropy,
#               metrics = ['accuracy'])
# train_feature_dict = {'creative_id':creative_id_train_seq,
#                 'ad_id':ad_id_train_seq}
# val_feature_dict = {'creative_id':creative_id_val_seq,
#                     'ad_id':ad_id_val_seq}
#train_dataset = input_fn(train_feature_dict, gender_train_label-1, epochs=5, shuffle=True, batch_size=128)
#val_dataset = input_fn(val_feature_dict, gender_val_label-1, epochs=1, shuffle=False, batch_size=128)
#model.fit(train_dataset, validation_data=val_dataset)
#model.save(model_path+'gender', save_format='tf')
#model.summary()

In [15]:
# tf.compat.v1.disable_eager_execution()
# estimator = tf.keras.estimator.model_to_estimator(model)
#estimator.train(input_fn=lambda : input_fn(train_feature_dict, gender_train_label-1, epochs=5, shuffle=True, batch_size=128)   )

INFO:tensorflow:Using default config.
INFO:tensorflow:Using the Keras model provided.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\ADMINI~1\\AppData\\Local\\Temp\\tmpxuwhm4zf', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


  
--旧版本--  



In [18]:
def input_fn(feature_dict, label=None, epochs=5, shuffle=True, batch_size=64, fit_key='train'):
    if fit_key == 'train':
        dataset = tf.data.Dataset.from_tensor_slices((feature_dict, label))
    else:
        dataset = tf.data.Dataset.from_tensor_slices((feature_dict))
    if shuffle:
        dataset = dataset.shuffle(100*batch_size)
    dataset = dataset.repeat(epochs).batch(batch_size)
    return dataset

In [20]:
class LSTModel(keras.Model):
    def __init__(self, units, num_classes, emb_size, max_len):
        super(LSTModel, self).__init__()
        self.units = units
        self.embedding1 = keras.layers.Embedding(creative_id_vs, 128, input_length=max_len, trainable=False, weights=[creative_id_em],mask_zero=True)
        self.embedding2 = keras.layers.Embedding(ad_id_vs, 128, input_length=max_len, trainable=False, weights=[ad_id_em],mask_zero=True)
        self.embedding3 = keras.layers.Embedding(advertiser_id_vs, 64, input_length=max_len, trainable=False, weights=[advertiser_id_em],mask_zero=True)
        self.embedding4 = keras.layers.Embedding(product_id_vs, 64, input_length=max_len, trainable=False, weights=[product_id_em],mask_zero=True)

        self.embedding5 = keras.layers.Embedding(industry_vs, 32, input_length=max_len, trainable=False, weights=[industry_em],mask_zero=True)
        self.embedding6 = keras.layers.Embedding(product_category_vs, 16, input_length=max_len, trainable=False, weights=[product_category_em],mask_zero=True)
        self.embedding7 = keras.layers.Embedding(click_times_vs, 8, input_length=max_len, trainable=False, weights=[click_times_em],mask_zero=True)
        self.embedding8 = keras.layers.Embedding(time_vs, 16, input_length=max_len, trainable=False, weights=[time_em],mask_zero=True)
        
        
        
        self.lstm = keras.layers.Bidirectional(keras.layers.LSTM(self.units))
        
        self.dense1 = keras.layers.Dense(self.units, activation='relu')
        self.dense2 = keras.layers.Dense(num_classes, activation='softmax')

    def call(self, feature_dict, training=None, mask=None):
        x1 = self.embedding1(feature_dict['creative_id'])
        x2 = self.embedding2(feature_dict['ad_id'])
        x3 = self.embedding3(feature_dict['advertiser_id'])
        x4 = self.embedding4(feature_dict['product_id'])
        
        x5 = self.embedding5(feature_dict['industry'])
        x6 = self.embedding6(feature_dict['product_category'])
        x7 = self.embedding7(feature_dict['click_times'])
        x8 = self.embedding8(feature_dict['time'])
        
        
        e = tf.concat([x1,x2, x3, x4, x5, x6, x7, x8],-1)
        x = self.lstm(e)
        #x = self.embedding(x)
        #x = self.lstm(x)
        x = self.dense1(x)
        x = self.dense2(x)
        return x

In [25]:
num_classes = 2
model = LSTModel(units=units,
                 num_classes=num_classes,
                 emb_size=embedding_dim, 
                 max_len=max_length)
model.compile(optimizer = keras.optimizers.Adam(0.001),
              loss = keras.losses.sparse_categorical_crossentropy,
              metrics = ['accuracy'])
train_feature_dict = {'creative_id':creative_id_train_seq,
                      'ad_id':ad_id_train_seq,
                      'advertiser_id':advertiser_id_train_seq,
                      'product_id':product_id_train_seq,
                     'industry':industry_train_seq,
                     'product_category':product_category_train_seq,
                     'click_times':click_times_train_seq,
                     'time':time_train_seq}

val_feature_dict = {'creative_id':creative_id_val_seq,
                    'ad_id':ad_id_val_seq,
                    'advertiser_id':advertiser_id_val_seq,
                    'product_id':product_id_val_seq,
                    'industry':industry_val_seq,
                     'product_category':product_category_val_seq,
                     'click_times':click_times_val_seq,
                     'time':time_val_seq
                   }
train_dataset = input_fn(train_feature_dict, gender_train_label-1, epochs=5, shuffle=True, batch_size=128)
val_dataset = input_fn(val_feature_dict, gender_val_label-1, epochs=1, shuffle=False, batch_size=128)
print('start!!')
model.fit(train_dataset, validation_data=val_dataset)
#model.save(model_path+'gender', save_format='tf')
model.summary()

start!!
Train for 31641 steps, validate for 704 steps
31641/31641 [==============================] - 3108s 98ms/step - loss: 0.1608 - accuracy: 0.9430 - val_loss: 0.1593 - val_accuracy: 0.9434
Model: "lst_model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_8 (Embedding)      multiple                  436834304 
_________________________________________________________________
embedding_9 (Embedding)      multiple                  387501952 
_________________________________________________________________
embedding_10 (Embedding)     multiple                  3703744   
_________________________________________________________________
embedding_11 (Embedding)     multiple                  2499712   
_________________________________________________________________
embedding_12 (Embedding)     multiple                  10656     
____________________________________________________________

In [21]:
num_classes = 10
units = 128
age_model = LSTModel(units=units,
                 num_classes=num_classes,
                 emb_size=embedding_dim, 
                 max_len=max_length)
age_model.compile(optimizer = keras.optimizers.Adam(0.001),
              loss = keras.losses.sparse_categorical_crossentropy,
              metrics = ['accuracy'])
train_feature_dict = {'creative_id':creative_id_train_seq,
                      'ad_id':ad_id_train_seq,
                      'advertiser_id':advertiser_id_train_seq,
                      'product_id':product_id_train_seq,
                     'industry':industry_train_seq,
                     'product_category':product_category_train_seq,
                     'click_times':click_times_train_seq,
                     'time':time_train_seq}

val_feature_dict = {'creative_id':creative_id_val_seq,
                    'ad_id':ad_id_val_seq,
                    'advertiser_id':advertiser_id_val_seq,
                    'product_id':product_id_val_seq,
                    'industry':industry_val_seq,
                     'product_category':product_category_val_seq,
                     'click_times':click_times_val_seq,
                     'time':time_val_seq
                   }
train_dataset = input_fn(train_feature_dict, age_train_label-1, epochs=5, shuffle=True, batch_size=128)
val_dataset = input_fn(val_feature_dict, age_val_label-1, epochs=1, shuffle=False, batch_size=128)
age_model.fit(train_dataset, validation_data=val_dataset)
#model.save(model_path+'age', save_format='tf')
age_model.summary()

Train for 31641 steps, validate for 704 steps
31641/31641 [==============================] - 3558s 112ms/step - loss: 1.3297 - accuracy: 0.4474 - val_loss: 1.3076 - val_accuracy: 0.4569
Model: "lst_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  436834304 
_________________________________________________________________
embedding_1 (Embedding)      multiple                  387501952 
_________________________________________________________________
embedding_2 (Embedding)      multiple                  3703744   
_________________________________________________________________
embedding_3 (Embedding)      multiple                  2499712   
_________________________________________________________________
embedding_4 (Embedding)      multiple                  10656     
_________________________________________________________________
emb

In [6]:
#gender_model = keras.models.load_model('gender')
#age_model = keras.models.load_model('age')


In [26]:
gender_pre = []
gender_prob = []
num_index = [0, 100000, 200000, 300000, 400000, 500000, 600000, 700000, 800000, 900000, 1000000]
for i in range(10):
    test_feature_dict = {'creative_id':creative_id_test_seq[num_index[i]:num_index[i+1]],
                    'ad_id':ad_id_test_seq[num_index[i]:num_index[i+1]],
                    'advertiser_id':advertiser_id_test_seq[num_index[i]:num_index[i+1]],
                    'product_id':product_id_test_seq[num_index[i]:num_index[i+1]],
                    'industry':industry_test_seq[num_index[i]:num_index[i+1]],
                     'product_category':product_category_test_seq[num_index[i]:num_index[i+1]],
                     'click_times':click_times_test_seq[num_index[i]:num_index[i+1]],
                     'time':time_test_seq[num_index[i]:num_index[i+1]]
                   }
    test_dataset = input_fn(test_feature_dict, epochs=1, shuffle=False, batch_size=1024, fit_key='predict')
    pre_tmp = model.predict(test_dataset)
    gender_pre.extend(pre_tmp.argmax(axis=1) + 1)
    gender_prob.extend(pre_tmp)
    print(i)

# test_feature_dict = {'creative_id':creative_id_test_seq,
#                     'ad_id':ad_id_test_seq}
# test_dataset = input_fn(test_feature_dict, epochs=1, shuffle=False, batch_size=8192, fit_key='predict')
# age_pre = model.predict(test_feature_dict)
gender_pre[0:10],len(gender_pre)

0
1
2
3
4
5
6
7
8
9


([1, 2, 2, 1, 1, 1, 1, 1, 1, 2], 1000000)

In [23]:
age_pre = []
age_prob = []
num_index = [0, 100000, 200000, 300000, 400000, 500000, 600000, 700000, 800000, 900000, 1000000]
for i in range(10):
    test_feature_dict = {'creative_id':creative_id_test_seq[num_index[i]:num_index[i+1]],
                    'ad_id':ad_id_test_seq[num_index[i]:num_index[i+1]],
                    'advertiser_id':advertiser_id_test_seq[num_index[i]:num_index[i+1]],
                    'product_id':product_id_test_seq[num_index[i]:num_index[i+1]],
                    'industry':industry_test_seq[num_index[i]:num_index[i+1]],
                     'product_category':product_category_test_seq[num_index[i]:num_index[i+1]],
                     'click_times':click_times_test_seq[num_index[i]:num_index[i+1]],
                     'time':time_test_seq[num_index[i]:num_index[i+1]]
                   }
    test_dataset = input_fn(test_feature_dict, epochs=1, shuffle=False, batch_size=1024, fit_key='predict')
    pre_tmp = age_model.predict(test_dataset)
    age_pre.extend(pre_tmp.argmax(axis=1) + 1)
    age_prob.extend(pre_tmp)
    print(i)

# test_feature_dict = {'creative_id':creative_id_test_seq,
#                     'ad_id':ad_id_test_seq}
# test_dataset = input_fn(test_feature_dict, epochs=1, shuffle=False, batch_size=8192, fit_key='predict')
# age_pre = model.predict(test_feature_dict)
age_pre[0:10],len(age_pre)

0
1
2
3
4
5
6
7
8
9


([3, 7, 2, 3, 4, 4, 9, 3, 2, 10], 1000000)

In [24]:
sub = pd.DataFrame()
sub['user_id'] = range(3000001,4000001)
sub['predicted_age'] = age_pre
#sub['predicted_gender'] = gender_pre
print('ok！')
sub.to_csv(f"{sub_path}/submission_8_input.csv", index=False, encoding='utf-8')

ok！


In [27]:
sub = pd.read_csv(f"{sub_path}/submission_8_input.csv", encoding='utf-8')
#sub['predicted_age'] = age_pre
sub['predicted_gender'] = gender_pre
print('ok！')

sub.to_csv(f"{sub_path}/submission_8_input.csv", index=False, encoding='utf-8')
sub.to_csv(f"{sub_path}/submission.csv", index=False, encoding='utf-8')

ok！


In [7]:
import numpy as np
import pandas as pd
a = np.array([[0.25,0.36], [0.44,0.88]])
b = np.array([[0.25,0.36], [0.44,0.88]])

p = []
p.extend(a)
p.extend(b)

dd = pd.DataFrame(p)
dd

,0,1
0,0.25,0.36
1,0.44,0.88
2,0.25,0.36
3,0.44,0.88
